In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
import random

In [2]:
df = pd.DataFrame()

In [3]:
df["x1"] = [1, 2, 3, 4, 5, 6, 6, 7, 9, 9]
df["x2"] = [5, 3, 6, 8, 1, 9, 5, 8, 9, 2]
df['label'] = [1, 1, 0, 1, 0, 1, 0, 1, 0, 0]

In [4]:
df.head()

,x1,x2,label
0,1,5,1
1,2,3,1
2,3,6,0
3,4,8,1
4,5,1,0


In [5]:
x = df.iloc[:, :2]
y = df.iloc[:, -1]
display(x, y)

,x1,x2
0,1,5
1,2,3
2,3,6
3,4,8
4,5,1
5,6,9
6,6,5
7,7,8
8,9,9
9,9,2


0    1
1    1
2    0
3    1
4    0
5    1
6    0
7    1
8    0
9    0
Name: label, dtype: int64

In [6]:
class Adaboost():
    def __init__(self, models):
        self.models = models
        self.alpha = [0] * models
        self.dts = []
        self.w = []
    
    def fit(self, x_train, y_train):
        self.w = [1/len(x_train)] * len(x_train)
        self.dts = [DecisionTreeClassifier(max_depth=1) for i in range(self.models)]
        
        for i in range(self.models):
            print("model ", i)
            self.dts[i].fit(x_train, y_train)
            y_pred = self.dts[i].predict(x_train)
            
            error = 0.00000000001
            for j in range(len(x_train)):
                if y_pred[j] != y_train[j]:
                    error = error + self.w[j]

            self.alpha[i] = 0.5 * np.log((1 - error)/error)
            for j in range(len(self.w)):
                if y_train[j] != y_pred[j]:
                    self.w[j] = self.w[j] + np.exp(self.alpha[i])
                else:
                    self.w[j] = self.w[j] + np.exp(-self.alpha[i])
            
            sum_weights = np.sum(self.w)
            for j in range(len(self.w)):
                self.w[j] = self.w[j]/sum_weights


            ranges = []
            ranges.append((0, self.w[0]))
            for j in range(1, len(x_train)):
                a = ranges[j-1][1]
                b = a + self.w[j]
                ranges.append((a, b))

            rows = []
            for j in range(0, len(x_train)):
                num = random.random()
                for k in range(0, len(ranges)):
                    if num >= ranges[k][0] and num < ranges[k][1]:
                        rows.append(k)

            x_train = x_train.iloc[rows, :]
            y_train = [y_train[row] for row in rows]

In [7]:
ad = Adaboost(3)
ad.fit(x, y)

model  0
model  1
model  2


In [8]:
print(ad.alpha)

[0.4236489301697922, 1.2616610969411173, 0.9719281512122633]
